### Welcome to the first version of my Python Clothing Generator

###### This is a program that creates shop-ready clothing designs. Specifically it does the following:

- Generates an image fpr the user in their chosen art style.
- Shows the generated images and prompts the user to continue (or to try again with a new generation).
- Removes the background of the generated image.
- Creates clothing mockups using the Printful API, allowing the user to search through the mockup database.


In [2]:
#!pip install rembg
import json
import os
from pathlib import Path
import openai
import requests
import base64
from base64 import b64decode
from pathlib import Path
import http.client
import shutil
import random
import sys 
import cv2
from PIL import Image
import collections
import re
import time




#Remove Background Imports
from rembg import remove
from PIL import Image


#Display Images
from io import BytesIO


C:\Users\User\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
def remove_background(input_path, saved_name):
    input_image = Image.open(input_path)
    output_path = Path.cwd() / "images" / "nobg" / saved_name
    output = remove(input_image)
    
    # Save the image in PNG format
    output_path = output_path.with_suffix('.png')
    output.save(output_path)
    
    return output_path, output


def upload_image_to_imgbb(image_path):
    url = "https://api.imgbb.com/1/upload"
    with open(image_path, "rb") as file:
        payload = {
            "key": "INSERT TOKEN HERE",
            "image": base64.b64encode(file.read()),
        }
    response = requests.post(url, payload)
    if response.status_code == 200:
        return response.json()["data"]["url"]
    else:
        print(f"Upload failed with status code {response.status_code}")
        return None

In [4]:
#HELPER FUNCTIONS:

def get_most_common_words(titles, n_most_common):
    words = []
    for title in titles:
        # Replace special characters with spaces
        title = re.sub(r'[^\w\s]', ' ', title)
        words.extend(title.lower().split())

    word_counter = collections.Counter(words)
    most_common_words = word_counter.most_common(n_most_common)

    return most_common_words


def string_to_list(s):
    return [int(x.strip()) for x in s.split(',')]


def display_image(path_or_url):
    if os.path.exists(path_or_url):
        # If it's a local file path
        img = Image.open(path_or_url)
    else:
        # If it's a URL, then fetch it using requests
        try:
            response = requests.get(path_or_url)
            img = Image.open(BytesIO(response.content))
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return None

    img.show()

In [5]:
    #Printful Store IDs
    
    oauth_token = "INSERT TOKEN HERE"  #Printful API Token
    
    url = 'https://api.printful.com/stores'

    headers = {
        "Authorization": f"Bearer {oauth_token}",
        "X-PF-Store-Id": "10298177"
    }
    

In [6]:
#Search for Product options on Printful

def mockup_info ():

    template_id = ""  
    oauth_token = "INSERT TOKEN HERE"  #Printful API Token

    url = 'https://api.printful.com/products'

    headers = {
        "Authorization": f"Bearer {oauth_token}"
    }

    response = requests.get(url, headers=headers)

    data = response.json()
    #print(data)
    
    filtered_products = []

    #start by returning all unique types
    types = set(product['type'] for product in data['result'])
    print(types)
    
    TYPE = input("Please select a mockup type from the list: ").upper()
    
    # Here's where we filter for products of a specific type
    product_filter = [product for product in data['result'] if product['type'] == TYPE]
    
    # Filter out unnecessary fields
    for product in product_filter:
        filtered_product = {
            'id': product['id'],
            'main_category_id': product['main_category_id'],
            'type': product['type'],
            'description': product['description'],
            'title': product['title'],
            'brand': product['brand'],
            'model': product['model'],
            'image': product['image'],
            'options': product['options'],
        }
        filtered_products.append(filtered_product)

    # Get all titles
    titles = [product['title'] for product in filtered_products]
    
    # Get most common words in titles
    most_common_words = get_most_common_words(titles, 20)  # Here 20 is the number of most common words you want to get
    print("\nMost common words in titles:")
    for word, count in most_common_words:
        print(f"{word}: {count}")

        
        
    title_search = input("Please provide the \"title\" to search by: ")    
    filtered_products = [product for product in filtered_products if title_search.lower() in product['title'].lower()]
    
    print(json.dumps(filtered_products, indent=2))  # Print filtered products not product_filter

#mockup_info()

In [7]:
def variant_IDs(product_id):  
    oauth_token = "INSERT TOKEN HERE"  #Printful API Token

    url = 'https://api.printful.com/products/'+product_id

    headers = {
        "Authorization": f"Bearer {oauth_token}"
        }

    response = requests.get(url, headers=headers)

    data = response.json()


    variant_info = data.get('result', {}).get('variants', {})

    for variant in variant_info:
        if not variant['in_stock']:
            continue
        print(f"Product ID: {variant['product_id']}")
        print(f"Variant ID: {variant['id']}")
        print(f"Name: {variant['name']}")
        print(f"Size: {variant['size']}")
        print(f"Color: {variant['color']}")
        print(f"Price: {variant['price']}")
        print(f"In Stock: {variant['in_stock']}")
        print(f"Availability Regions: {variant['availability_regions']}")
        print(f"Availability Status: {variant['availability_status']}")
        print(f"Material: {variant['material']}")
        print("-"*50)  # separator for readability

    return (input("Add any desired \"variant IDs\" seperated by commas: "))


In [8]:
def mockup_status(task_key):
    
    oauth_token = "INSERT TOKEN HERE" 
    
    url = "https://api.printful.com/mockup-generator/task?task_key="+task_key

    headers = {
        "Authorization": f"Bearer {oauth_token}",
        "X-PF-Store-Id": "10298177",
    }

    while True:
        response = requests.get(url, headers=headers)
        data = response.json()
        status = data['result']['status']
        
        if status == 'completed':
            print(f"Task Key: {data['result']['task_key']}")
            print(f"Status: {data['result']['status']}")
            for mockup in data['result']['mockups']:
                print(f"\nMockup Details:")
                print(f"\tPlacement: {mockup['placement']}")
                print(f"\tVariant IDs: {mockup['variant_ids']}")
                print(f"\tMockup URL: {mockup['mockup_url']}")
                for extra in mockup['extra']:
                    print(f"\nExtra Details:")
                    print(f"\tTitle: {extra['title']}")
                    print(f"\tOption: {extra['option']}")
                    print(f"\tOption Group: {extra['option_group']}")
                    print(f"\tURL: {extra['url']}")
            for printfile in data['result']['printfiles']:
                print(f"\nPrintfile Details:")
                print(f"\tVariant IDs: {printfile['variant_ids']}")
                print(f"\tPlacement: {printfile['placement']}")
                print(f"\tURL: {printfile['url']}")
            break  # Exit the loop if status is completed
        else:
            print(f"Status: {status}. Retrying in 10 seconds...")
            time.sleep(10)  # Wait for 10 seconds before the next attempt
#mockup_status('gt-533413472')

In [9]:
def mockup_generator(product_id,image_url):
    
    oauth_token = "INSERT TOKEN HERE"  #Printful API Token
    
    chosen_ids = string_to_list(variant_IDs(product_id))
    print(chosen_ids)
    
    url = "https://api.printful.com/mockup-generator/create-task/"+product_id

    headers = {
        "Authorization": f"Bearer {oauth_token}",
        "X-PF-Store-Id": "10298177",
        "Content-Type": "application/json"
    }
    payload = {
        "variant_ids": chosen_ids,
        "format": "jpg",
        "files": [
    {
      "placement": "front",
      "image_url": image_url,
      "position": {
        "area_width": 1800,
        "area_height": 2400,
        "width": 1800,
        "height": 1800,
        "top": 300,
        "left": 0
      }
    }]
    }
    
    
    response = requests.post(url, headers=headers, json=payload)

    data = response.json()
    task_key = data['result']['task_key'] # Extracting the task_key from the response
    print("Mockup Generation started at Task Key #"+task_key)
    
    mockup_status(task_key)
    
#mockup_generator("318", image_url)

In [10]:
def image_generator():
    CONTINUE = True
    while CONTINUE == True:
        #User Inputs
        PROMPT = input("provide your image prompt: \n")
        MODEL = input("Pick a model: \"anime\", \"vector\", or \"realism\" or \"midjourney\" or \"shirt\"\n").lower()

        #Determine the model
        if MODEL not in ["anime", "vector", "realism", "midjourney", "shirt"]:
            print("incorrect model")
            return

        elif MODEL == "anime":
            model = "anything-v4"
        elif MODEL == "vector":
            model = "vector-art"
        elif MODEL == "realism":
            model = "protogen-3.4"
        elif MODEL == "midjourney":
            model = "midjourney"
        elif MODEL =="shirt":
            model = "t-shirt-prin"
        print("generating with " + model)  



        openai.api_key = "INSERT KEY HERE"

        DATA_DIR = Path.cwd() / "images"
        DATA_DIR.mkdir(exist_ok=True)

        conn = http.client.HTTPSConnection("stablediffusionapi.com")
        payload = {
            "key": "INSERT KEY HERE",
            "prompt": "(("+PROMPT+")),vivid colors, colorful, pastel cute colors, ((vector-art),(centered and enclosed on a black background:1.2) as a t-shirt logo",
            "model_id": model,
            "negative_prompt": "((watermark)), ((signature)),low poly, tetric, mosaic, disfigured, kitsch, ugly, oversaturated, grain, low-res, Deformed, blurry, bad anatomy, disfigured, poorly drawn face, mutation, mutated, extra limb, ugly, poorly drawn hands, missing limb, blurry, floating limbs, disconnected limbs, malformed hands, blur, out of focus, long neck, long body, ugly, disgusting, poorly drawn, mutilated, mangled, old, surreal, pixel-art, black and white, childish",
            "width": "512",
            "height": "512",
            "samples": "1",
            "num_inference_steps": "50",
            "seed": "",
            "guidance_scale": "7",
            "webhook": "",
            "track_id": ""
        }

        headers = {"Content-Type": "application/json"}  # You may need to adjust this line based on the API's documentation

        conn.request("POST", "/api/v3/text2img", json.dumps(payload), headers)
        res = conn.getresponse()
        data = res.read()
        data_str = data.decode("utf-8")

        data_dict = json.loads(data_str)
        #print(data_dict)
        image_urls = data_dict["output"] 

        # Download the first image to a local file
        image_url = image_urls[0]
        image_file = Path.cwd() / "images" / "bg" / f"{PROMPT}{random.randint(0,999999)}.jpg"  # use the prompt as the filename
        saved_name = f"{PROMPT}{random.randint(0,999999)}.jpg"
        print("Generated " + saved_name)

        
        #Check image to continue
        display_image(image_url)

        if input("continue? yes or no?\n") == "no":
            continue
        
        # Download the image
        print("Downloading the Image...")    
        response = requests.get(image_url, stream=True)
        with open(image_file, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response
        
        #Remove the Background
        print("Removing BG...")
        image_path, image_file = remove_background(image_file, saved_name)
        display_image(image_path)

        if input("continue? yes or no?\n") == "no":
            continue  
        
        #Uploading image to imgbb 
        print("uploading the image to imgbb")
        image_url = upload_image_to_imgbb(image_path)
        
        #Generate Mockups - start by letting the user search for mockups and variants to generate

        if input("Proceed with Mockup Search? yes or no? If you do not have your product ID ready, please input \"yes\": ") == "yes":
            mockup_info()
            

        mockup_generator(input("Please provide Product ID: "),image_url)

        print("Done!") 
        if input("Continue? Yes or No \n").lower() == "no":
            print("Exiting")
            CONTINUE = False
            break
                
            

In [11]:

image_generator()

provide your image prompt: 
Test
Pick a model: "anime", "vector", or "realism" or "midjourney" or "shirt"
Anime
generating with anything-v4
Generated Test786400.jpg
continue? yes or no?
yes
Removing BG...
continue? yes or no?
yes
uploading the image to imgbb
Proceed with Mockup Search? yes or no? If you do not have your product ID ready, please input "yes": no
Please provide Product ID: 521
Product ID: 521
Variant ID: 13118
Name: United Athle 5214-01 (Black / L)
Size: L
Color: Black
Price: 25.75
In Stock: True
Availability Regions: {'JP': 'Japan'}
Availability Status: [{'region': 'JP', 'status': 'in_stock'}]
Material: [{'name': 'Cotton', 'percentage': 100}]
--------------------------------------------------
Product ID: 521
Variant ID: 13117
Name: United Athle 5214-01 (Black / M)
Size: M
Color: Black
Price: 25.75
In Stock: True
Availability Regions: {'JP': 'Japan'}
Availability Status: [{'region': 'JP', 'status': 'in_stock'}]
Material: [{'name': 'Cotton', 'percentage': 100}]
-----------

Add any desired "variant IDs" seperated by commas: 13155
[13155]
Mockup Generation started at Task Key #gt-549305867
Status: pending. Retrying in 10 seconds...
Task Key: gt-549305867
Status: completed

Mockup Details:
	Placement: front
	Variant IDs: [13155]
	Mockup URL: https://printful-upload.s3-accelerate.amazonaws.com/tmp/6f84966adc26726429b8fab24bc47811/unisex-french-terry-pullover-hoodie-white-front-64bf18a3a9c9e.jpg

Extra Details:
	Title: Front 2
	Option: Front 2
	Option Group: Men's
	URL: https://printful-upload.s3-accelerate.amazonaws.com/tmp/65df1d0c424760b7387d91086b420206/unisex-french-terry-pullover-hoodie-white-front-2-64bf18a3aabb9.jpg

Printfile Details:
	Variant IDs: [13155]
	Placement: front
	URL: https://printful-upload.s3-accelerate.amazonaws.com/tmp/3351359e175de27c2a82f760aa95dae3/printfile_front.png
Done!
Continue? Yes or No 
no
Exiting
